In [1]:
import sys
sys.path.append('..')
import torch
import torch.backends.cudnn as cudnn
import matplotlib.pyplot as plt
from src.training import train_model, eval_func
import os
from torchvision import datasets, models, transforms
from src.data_loader import imshow
from src.modelvis import visualize_models
from torch.optim.lr_scheduler import StepLR
import torchvision
import torch.nn as nn
'''
The line cudnn.benchmark = True is typically used in deep learning projects that utilize the CUDA Deep Neural Network (cuDNN) library.
When cudnn.benchmark is set to True, it enables cuDNN to automatically find the best algorithm configuration for the specific 
input sizes and hardware being used. This can result in improved performance during training and inference.By enabling benchmarking, 
cuDNN will run a short benchmarking phase during the first iteration of the model to determine the optimal algorithm configuration. 
This configuration is then cached and used for subsequent iterations, leading to faster execution times.
It's important to note that enabling benchmarking may introduce some overhead during the initial benchmarking phase, so it is typically 
recommended to use it when the input sizes are consistent throughout the training process.
Overall, setting cudnn.benchmark to True can help optimize the performance of deep learning models that use cuDNN.
'''
# cudnn.benchmark = True

'''
The line plt.ion() is a function call that activates interactive mode in matplotlib.
When interactive mode is enabled, any plot that is created will be displayed immediately 
and can be updated dynamically. This means that you can modify the plot after it is displayed, 
such as changing the data or adding annotations, and the changes will be reflected in real-time.
'''
plt.ion()

Device: cuda:0


In [2]:
print(torchvision.__version__)
print(torch.__version__)

0.16.1+cu118
2.1.1+cu118


In [3]:
# timm.list_models(pretrained=True)
from torchvision.models import vgg11, VGG11_Weights

class VGG11_FC_Changed(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        weights = VGG11_Weights.DEFAULT
        self.model = vgg11(weights=weights, progress=False)
        self.model.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(True),
            nn.Dropout(p=0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(p=0.5),
            nn.Linear(4096, num_classes),
        )
        self.transform = weights.transforms(antialias=True)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.transform(x)
        return self.model(x)


In [4]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(84),
        # transforms.RandomResizedCrop(84),
        transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.GaussianBlur(3,sigma=(0.5, 2.0)),
        transforms.RandomRotation(degrees=(0, 30)),
        transforms.RandomAdjustSharpness(0.25),
        transforms.RandomAutocontrast(0.25),
        transforms.RandomEqualize(),
        transforms.ToTensor(),
    ]),
    'tune': transforms.Compose([
        transforms.Resize(224),
        transforms.RandomAdjustSharpness(0.25),
        transforms.RandomAutocontrast(0.25),
        transforms.RandomEqualize(),
        transforms.ToTensor(),
    ]),
    'test': transforms.Compose([
        transforms.ToTensor(),
    ]),
}

def GetDataLoaders(data_dir, batch_size=4, shuffle=True, num_workers=4, validation_split=0.2, test_split=0.1):
    original_train_dataset = datasets.ImageFolder(data_dir, data_transforms['train'])

    num_samples = len(original_train_dataset)
    num_val = int(validation_split * num_samples)
    num_test = int(test_split * num_samples)
    num_train = num_samples - num_val - num_test

    # Split the dataset into train, validation, and test sets
    train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(original_train_dataset, [num_train, num_val, num_test])

    # Create data loaders for each set
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers)
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers)

    dataloaders = {
        'train': train_loader,
        'val': val_loader,
        'test': test_loader
    }
    dataset_sizes = {
        'train': num_train,
        'val': num_val,
        'test': num_test
    }
    class_names = original_train_dataset.classes

    return dataloaders, class_names, dataset_sizes

### Hyperparameters

In [5]:
T_0 = 1000 # Number of iterations for the first restart.
LEARNING_RATE=0.001 # 0.0001
WEIGHT_DECAY=0.0005 # 0.000001
NUM_EPOCHS=15
BATCH_SIZE=64
MOMENTUM=0.9
num_of_classes = 64
STEP_SIZE=7
GAMMA=0.01

### Data loading parameters

In [6]:
SHUFFLE=True
WORKERS=16
num_images = 8

In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

data_dir = '..\\data\\miniImageNet\\train'
dataloaders, class_names, dataset_sizes  = GetDataLoaders(data_dir, BATCH_SIZE, SHUFFLE, num_workers=WORKERS)

train_loader = dataloaders['train']
validation_loader = dataloaders['val']
test_loader = dataloaders['test']

In [8]:
criterion = torch.nn.CrossEntropyLoss()

In [ ]:
net = VGG11_FC_Changed(num_of_classes).to(device)
for p in net.parameters():
    p.requires_grad = True
net.train()
optimizer = torch.optim.SGD(params= net.parameters(), lr=LEARNING_RATE, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)
scheduler = StepLR(optimizer, step_size=STEP_SIZE, gamma=GAMMA)
print('Please wait patiently, it may take some seconds...')
best_model = train_model(net, dataloaders, criterion, optimizer, scheduler, NUM_EPOCHS, dataset_sizes)
save_path = '..\\data\\models\\best_model_vgg11.pth'
torch.save(best_model.state_dict(), save_path)

eval_acc = eval_func(best_model, dataloaders['test'])
print('')
print('Accuracy on testing data: %f' % eval_acc)

Downloading: "https://download.pytorch.org/models/vgg11-8a719046.pth" to C:\Users\rperera23/.cache\torch\hub\checkpoints\vgg11-8a719046.pth


Please wait patiently, it may take some seconds...
Epoch 0/14
----------
train Loss: 0.0595 Acc: 0.0834
val Loss: 0.0446 Acc: 0.2796

Epoch 1/14
----------
train Loss: 0.0418 Acc: 0.2996


In [ ]:
best_model.load_state_dict(torch.load(save_path))
visualize_models(best_model, dataloaders, num_images, class_names)